In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import  seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score

/home/paul/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#### Loading data

In [2]:
train_le = pd.read_csv('dataset/hackathon.csv/Train.csv')
test_le = pd.read_csv('dataset/hackathon.csv/Test.csv')

In [3]:
# Converting Categorical datas to numerical data and dropping col Applicant_ID
c_l = pd.get_dummies(train_le['form_field47'],drop_first=True)
d_s = pd.get_dummies(train_le['default_status'],drop_first=True)

train_le = pd.concat([train_le,c_l,d_s],axis=1)
train_le.drop(['form_field47','default_status','Applicant_ID'],axis=1,inplace=True)

In [4]:
c_l = pd.get_dummies(test_le['form_field47'],drop_first=True)
# d_s = pd.get_dummies(train['default_status'],drop_first=True)

test_le = pd.concat([test_le,c_l],axis=1)
test_le.drop(['form_field47','Applicant_ID'],axis=1,inplace=True)


In [5]:
# Independent Features / Dependent Features
X = train_le.drop(['yes'],axis=1)
y = train_le['yes']

In [6]:
test_le.fillna(-999,inplace=True)

In [7]:
X.fillna(-999,inplace=True)

In [5]:
# # Encoding categorical data
# # Encoding the Independent Variable
# from sklearn.preprocessing import LabelEncoder
# labelencoder_X = LabelEncoder()
# X.iloc[:, 46] = labelencoder_X.fit_transform(X.iloc[:, 46])
# # Encoding the Dependent Variable
# labelencoder_y = LabelEncoder()
# y = labelencoder_y.fit_transform(y)

In [6]:
# # Encoding categorical data
# # Encoding the Independent Variable
# from sklearn.preprocessing import LabelEncoder
# labelencoder_test_le = LabelEncoder()
# test_le.iloc[:, 46] = labelencoder_test_le.fit_transform(test_le.iloc[:, 46])
# # Encoding the Dependent Variable

Note:

1.I came to understand that StratifiedKFold work better on imbalance data like our dataset

2.From the previous prediction i submitted which comprises of different learning model sure as neural network,svm,xgboost,lightgbm and more i discover that catboost classifier boosted my roc_auc_sroce signficantly well, with the depth of 6,learing_rate of 0.037,n_estimator of 700

3.I also made some reaseacrh on VotingClassifier and applied some model that actually heard a better roc_auc_score

In [8]:
rf = RandomForestClassifier(n_estimators=500)
lg = LGBMClassifier()
cb = CatBoostClassifier(depth=6,learning_rate=0.037,n_estimators=700)
cb1 = CatBoostClassifier()

In [9]:
classifiers = [('rf',rf),('lg',lg),('cb',cb),('cb1',cb1)]

In [10]:
skf = StratifiedKFold(n_splits=30,shuffle=True,random_state=0)
# Taking the score for each validation for a partically KFold 
scores = []
# Taking the predicted value of each test.csv row 
preds = []
vc = VotingClassifier(estimators=classifiers,voting='soft',
                     verbose=False)


i = 1
for train,test in skf.split(X,y):
    pipe = Pipeline([('scaler',MinMaxScaler()),('model',vc)])
    X_train,X_test,y_train,y_test = X.iloc[train],X.iloc[test],y[train],y[test]
    pipe.fit(X_train,y_train)
    print('Numbers of Splits trained {}'.format(i))
    score = roc_auc_score(y_test,pipe.predict_proba(X_test)[:,1])
    pred = pipe.predict_proba(test_le)[:,1]
    scores.append(score)
    preds.append(pred)
    i += 1
# Taking the average of all the Score in each split
print(np.mean(scores))
# Taking the average of all the predicted value on the test.csv dataset
final_pred = np.mean(preds,axis=0)


0:	learn: 0.6698566	total: 82.1ms	remaining: 57.4s
1:	learn: 0.6484323	total: 108ms	remaining: 37.6s
2:	learn: 0.6286093	total: 139ms	remaining: 32.3s
3:	learn: 0.6115282	total: 169ms	remaining: 29.4s
4:	learn: 0.5958406	total: 200ms	remaining: 27.8s
5:	learn: 0.5820717	total: 229ms	remaining: 26.5s
6:	learn: 0.5687246	total: 255ms	remaining: 25.3s
7:	learn: 0.5567628	total: 286ms	remaining: 24.7s
8:	learn: 0.5462865	total: 325ms	remaining: 25s
9:	learn: 0.5367682	total: 355ms	remaining: 24.5s
10:	learn: 0.5273991	total: 384ms	remaining: 24.1s
11:	learn: 0.5188560	total: 413ms	remaining: 23.7s
12:	learn: 0.5118209	total: 438ms	remaining: 23.2s
13:	learn: 0.5056753	total: 464ms	remaining: 22.7s
14:	learn: 0.4996900	total: 491ms	remaining: 22.4s
15:	learn: 0.4934521	total: 529ms	remaining: 22.6s
16:	learn: 0.4883999	total: 556ms	remaining: 22.4s
17:	learn: 0.4833206	total: 582ms	remaining: 22.1s
18:	learn: 0.4787026	total: 608ms	remaining: 21.8s
19:	learn: 0.4746418	total: 633ms	remainin

163:	learn: 0.4022447	total: 4.86s	remaining: 15.9s
164:	learn: 0.4021572	total: 4.89s	remaining: 15.9s
165:	learn: 0.4020415	total: 4.92s	remaining: 15.8s
166:	learn: 0.4019783	total: 4.95s	remaining: 15.8s
167:	learn: 0.4019048	total: 4.98s	remaining: 15.8s
168:	learn: 0.4018121	total: 5.02s	remaining: 15.8s
169:	learn: 0.4017440	total: 5.06s	remaining: 15.8s
170:	learn: 0.4016820	total: 5.1s	remaining: 15.8s
171:	learn: 0.4015824	total: 5.13s	remaining: 15.7s
172:	learn: 0.4014734	total: 5.16s	remaining: 15.7s
173:	learn: 0.4014189	total: 5.19s	remaining: 15.7s
174:	learn: 0.4013613	total: 5.22s	remaining: 15.7s
175:	learn: 0.4012534	total: 5.26s	remaining: 15.7s
176:	learn: 0.4011709	total: 5.3s	remaining: 15.7s
177:	learn: 0.4010706	total: 5.33s	remaining: 15.6s
178:	learn: 0.4009676	total: 5.36s	remaining: 15.6s
179:	learn: 0.4008718	total: 5.39s	remaining: 15.6s
180:	learn: 0.4007843	total: 5.42s	remaining: 15.5s
181:	learn: 0.4007246	total: 5.45s	remaining: 15.5s
182:	learn: 0.

324:	learn: 0.3909524	total: 10.1s	remaining: 11.7s
325:	learn: 0.3908916	total: 10.2s	remaining: 11.7s
326:	learn: 0.3908294	total: 10.2s	remaining: 11.6s
327:	learn: 0.3907694	total: 10.2s	remaining: 11.6s
328:	learn: 0.3906998	total: 10.3s	remaining: 11.6s
329:	learn: 0.3906270	total: 10.3s	remaining: 11.5s
330:	learn: 0.3905428	total: 10.3s	remaining: 11.5s
331:	learn: 0.3904637	total: 10.3s	remaining: 11.4s
332:	learn: 0.3903694	total: 10.4s	remaining: 11.4s
333:	learn: 0.3903015	total: 10.4s	remaining: 11.4s
334:	learn: 0.3902183	total: 10.4s	remaining: 11.4s
335:	learn: 0.3901529	total: 10.5s	remaining: 11.3s
336:	learn: 0.3901101	total: 10.5s	remaining: 11.3s
337:	learn: 0.3900293	total: 10.5s	remaining: 11.3s
338:	learn: 0.3899730	total: 10.5s	remaining: 11.2s
339:	learn: 0.3899193	total: 10.6s	remaining: 11.2s
340:	learn: 0.3898632	total: 10.6s	remaining: 11.1s
341:	learn: 0.3898120	total: 10.6s	remaining: 11.1s
342:	learn: 0.3897427	total: 10.6s	remaining: 11.1s
343:	learn: 

490:	learn: 0.3794357	total: 14.7s	remaining: 6.24s
491:	learn: 0.3793921	total: 14.7s	remaining: 6.21s
492:	learn: 0.3793464	total: 14.7s	remaining: 6.18s
493:	learn: 0.3792720	total: 14.7s	remaining: 6.15s
494:	learn: 0.3791837	total: 14.8s	remaining: 6.12s
495:	learn: 0.3791327	total: 14.8s	remaining: 6.09s
496:	learn: 0.3790865	total: 14.8s	remaining: 6.06s
497:	learn: 0.3790242	total: 14.9s	remaining: 6.04s
498:	learn: 0.3789554	total: 14.9s	remaining: 6.01s
499:	learn: 0.3788809	total: 15s	remaining: 5.99s
500:	learn: 0.3788152	total: 15s	remaining: 5.96s
501:	learn: 0.3787492	total: 15s	remaining: 5.93s
502:	learn: 0.3786917	total: 15.1s	remaining: 5.9s
503:	learn: 0.3786185	total: 15.1s	remaining: 5.88s
504:	learn: 0.3785688	total: 15.1s	remaining: 5.84s
505:	learn: 0.3784974	total: 15.2s	remaining: 5.81s
506:	learn: 0.3784252	total: 15.2s	remaining: 5.78s
507:	learn: 0.3783709	total: 15.2s	remaining: 5.75s
508:	learn: 0.3783011	total: 15.2s	remaining: 5.72s
509:	learn: 0.37822

656:	learn: 0.3693925	total: 19s	remaining: 1.24s
657:	learn: 0.3693296	total: 19s	remaining: 1.21s
658:	learn: 0.3692754	total: 19s	remaining: 1.19s
659:	learn: 0.3692099	total: 19.1s	remaining: 1.16s
660:	learn: 0.3691431	total: 19.1s	remaining: 1.13s
661:	learn: 0.3690900	total: 19.1s	remaining: 1.1s
662:	learn: 0.3690662	total: 19.1s	remaining: 1.07s
663:	learn: 0.3690100	total: 19.2s	remaining: 1.04s
664:	learn: 0.3689631	total: 19.2s	remaining: 1.01s
665:	learn: 0.3689599	total: 19.2s	remaining: 981ms
666:	learn: 0.3688949	total: 19.2s	remaining: 952ms
667:	learn: 0.3688500	total: 19.3s	remaining: 923ms
668:	learn: 0.3687806	total: 19.3s	remaining: 894ms
669:	learn: 0.3687389	total: 19.3s	remaining: 865ms
670:	learn: 0.3686810	total: 19.3s	remaining: 836ms
671:	learn: 0.3686350	total: 19.4s	remaining: 807ms
672:	learn: 0.3685726	total: 19.4s	remaining: 778ms
673:	learn: 0.3685085	total: 19.4s	remaining: 749ms
674:	learn: 0.3684578	total: 19.4s	remaining: 720ms
675:	learn: 0.36840

...

290:	learn: 0.3836629	total: 7.58s	remaining: 18.5s
291:	learn: 0.3835717	total: 7.61s	remaining: 18.4s
292:	learn: 0.3835001	total: 7.63s	remaining: 18.4s
293:	learn: 0.3833851	total: 7.66s	remaining: 18.4s
294:	learn: 0.3832964	total: 7.68s	remaining: 18.4s
295:	learn: 0.3831484	total: 7.71s	remaining: 18.3s
296:	learn: 0.3830661	total: 7.74s	remaining: 18.3s
297:	learn: 0.3829344	total: 7.76s	remaining: 18.3s
298:	learn: 0.3828425	total: 7.79s	remaining: 18.3s
299:	learn: 0.3827382	total: 7.82s	remaining: 18.3s
300:	learn: 0.3826165	total: 7.85s	remaining: 18.2s
301:	learn: 0.3825466	total: 7.87s	remaining: 18.2s
302:	learn: 0.3824219	total: 7.9s	remaining: 18.2s
303:	learn: 0.3823611	total: 7.92s	remaining: 18.1s
304:	learn: 0.3822685	total: 7.95s	remaining: 18.1s
305:	learn: 0.3821534	total: 7.97s	remaining: 18.1s
306:	learn: 0.3820380	total: 8s	remaining: 18.1s
307:	learn: 0.3819280	total: 8.03s	remaining: 18s
308:	learn: 0.3818533	total: 8.05s	remaining: 18s
309:	learn: 0.381757

456:	learn: 0.3686893	total: 11.9s	remaining: 14.1s
457:	learn: 0.3685904	total: 11.9s	remaining: 14.1s
458:	learn: 0.3685163	total: 11.9s	remaining: 14s
459:	learn: 0.3684236	total: 11.9s	remaining: 14s
460:	learn: 0.3683258	total: 12s	remaining: 14s
461:	learn: 0.3682412	total: 12s	remaining: 14s
462:	learn: 0.3681700	total: 12.1s	remaining: 14s
463:	learn: 0.3680826	total: 12.1s	remaining: 14s
464:	learn: 0.3679980	total: 12.1s	remaining: 13.9s
465:	learn: 0.3679160	total: 12.1s	remaining: 13.9s
466:	learn: 0.3677996	total: 12.2s	remaining: 13.9s
467:	learn: 0.3677206	total: 12.2s	remaining: 13.9s
468:	learn: 0.3676349	total: 12.2s	remaining: 13.9s
469:	learn: 0.3675449	total: 12.3s	remaining: 13.9s
470:	learn: 0.3674832	total: 12.3s	remaining: 13.8s
471:	learn: 0.3673999	total: 12.4s	remaining: 13.8s
472:	learn: 0.3673000	total: 12.4s	remaining: 13.8s
473:	learn: 0.3672282	total: 12.4s	remaining: 13.8s
474:	learn: 0.3672142	total: 12.5s	remaining: 13.8s
475:	learn: 0.3671045	total:

619:	learn: 0.3559879	total: 16.8s	remaining: 10.3s
620:	learn: 0.3558958	total: 16.8s	remaining: 10.2s
621:	learn: 0.3558932	total: 16.8s	remaining: 10.2s
622:	learn: 0.3557957	total: 16.8s	remaining: 10.2s
623:	learn: 0.3557279	total: 16.9s	remaining: 10.2s
624:	learn: 0.3556584	total: 16.9s	remaining: 10.1s
625:	learn: 0.3555807	total: 16.9s	remaining: 10.1s
626:	learn: 0.3554808	total: 16.9s	remaining: 10.1s
627:	learn: 0.3554034	total: 17s	remaining: 10s
628:	learn: 0.3553220	total: 17s	remaining: 10s
629:	learn: 0.3552322	total: 17s	remaining: 9.99s
630:	learn: 0.3551378	total: 17s	remaining: 9.97s
631:	learn: 0.3550756	total: 17.1s	remaining: 9.94s
632:	learn: 0.3549933	total: 17.1s	remaining: 9.91s
633:	learn: 0.3549064	total: 17.1s	remaining: 9.88s
634:	learn: 0.3548399	total: 17.1s	remaining: 9.85s
635:	learn: 0.3547660	total: 17.2s	remaining: 9.83s
636:	learn: 0.3546589	total: 17.2s	remaining: 9.8s
637:	learn: 0.3545564	total: 17.2s	remaining: 9.77s
638:	learn: 0.3544822	tot

780:	learn: 0.3446207	total: 20.9s	remaining: 5.86s
781:	learn: 0.3445318	total: 20.9s	remaining: 5.84s
782:	learn: 0.3444410	total: 21s	remaining: 5.81s
783:	learn: 0.3443366	total: 21s	remaining: 5.78s
784:	learn: 0.3442700	total: 21s	remaining: 5.76s
785:	learn: 0.3442112	total: 21s	remaining: 5.73s
786:	learn: 0.3441295	total: 21.1s	remaining: 5.7s
787:	learn: 0.3440641	total: 21.1s	remaining: 5.67s
788:	learn: 0.3439893	total: 21.1s	remaining: 5.65s
789:	learn: 0.3439417	total: 21.1s	remaining: 5.62s
790:	learn: 0.3438663	total: 21.2s	remaining: 5.59s
791:	learn: 0.3438142	total: 21.2s	remaining: 5.57s
792:	learn: 0.3437497	total: 21.2s	remaining: 5.54s
793:	learn: 0.3436737	total: 21.3s	remaining: 5.51s
794:	learn: 0.3435882	total: 21.3s	remaining: 5.49s
795:	learn: 0.3435150	total: 21.3s	remaining: 5.46s
796:	learn: 0.3434474	total: 21.3s	remaining: 5.43s
797:	learn: 0.3433684	total: 21.4s	remaining: 5.41s
798:	learn: 0.3433054	total: 21.4s	remaining: 5.38s
799:	learn: 0.3432149

939:	learn: 0.3341912	total: 25s	remaining: 1.6s
940:	learn: 0.3341259	total: 25s	remaining: 1.57s
941:	learn: 0.3340686	total: 25.1s	remaining: 1.54s
942:	learn: 0.3340071	total: 25.1s	remaining: 1.52s
943:	learn: 0.3339296	total: 25.1s	remaining: 1.49s
944:	learn: 0.3338776	total: 25.1s	remaining: 1.46s
945:	learn: 0.3338531	total: 25.2s	remaining: 1.44s
946:	learn: 0.3337694	total: 25.2s	remaining: 1.41s
947:	learn: 0.3337248	total: 25.2s	remaining: 1.38s
948:	learn: 0.3336580	total: 25.2s	remaining: 1.36s
949:	learn: 0.3335998	total: 25.3s	remaining: 1.33s
950:	learn: 0.3335301	total: 25.3s	remaining: 1.3s
951:	learn: 0.3334729	total: 25.3s	remaining: 1.28s
952:	learn: 0.3333974	total: 25.3s	remaining: 1.25s
953:	learn: 0.3333288	total: 25.4s	remaining: 1.22s
954:	learn: 0.3333214	total: 25.4s	remaining: 1.2s
955:	learn: 0.3332338	total: 25.4s	remaining: 1.17s
956:	learn: 0.3331771	total: 25.4s	remaining: 1.14s
957:	learn: 0.3331170	total: 25.5s	remaining: 1.12s
958:	learn: 0.33305

### Submission

In [13]:
Ss = pd.read_csv('dataset/hackathon.csv/SampleSubmission.csv')
Ss['default_status'] = final_pred
Ss
# Ss.to_csv('submit_hackme2.csv',index=False)for submission

,Applicant_ID,default_status
0,Apcnt_1000032,0.321392
1,Apcnt_1000048,0.351381
2,Apcnt_1000052,0.391921
3,Apcnt_1000076,0.721086
4,Apcnt_1000080,0.179941
...,...,...
23995,Apcnt_999940,0.688250
23996,Apcnt_999956,0.295195
23997,Apcnt_999976,0.280073
23998,Apcnt_999984,0.540165
